In [1]:
import torch
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import Trainer, TrainingArguments

c:\Users\antip\studying\Sveta\.venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  

model_name_or_path = "sberbank-ai/rugpt3small_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path,
                                          cache_dir="rugpt3small_based_on_gpt2_cached_token/")
model = GPT2LMHeadModel.from_pretrained(model_name_or_path, pad_token_id=tokenizer.eos_token_id,
                                        cache_dir="rugpt3small_based_on_gpt2_cached_model/").to(DEVICE)



In [3]:
# Сохраним обучающие данные в .txt файл 
train_path = 'train.txt'
def load_doc(filename):
    file = open(filename, 'r', encoding='utf-8')
    text = file.read()
    file.close()
    return text

data = load_doc('book.txt')
with open(train_path, "w") as f:
    f.write(data)

# Создание датасета
train_dataset = TextDataset(tokenizer=tokenizer,file_path=train_path,block_size=64)
  
# Создание даталодера (нарезает текст на оптимальные по длине куски)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, 
                                                mlm=False)

c:\Users\antip\studying\Sveta\.venv\lib\site-packages\transformers\data\datasets\language_modeling.py:54: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [4]:
print(len(train_dataset))

2124


In [5]:
training_args = TrainingArguments(
    output_dir="./finetuned", 
    overwrite_output_dir=True, 
    num_train_epochs=50, 
    per_device_train_batch_size=16, 
    per_device_eval_batch_size=16,  
    warmup_steps=10, 
    gradient_accumulation_steps=8, 
    )

torch.cuda.set_per_process_memory_fraction(0.8, 0)
torch.cuda.empty_cache()

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    optimizers = (torch.optim.AdamW(model.parameters(),lr=1e-5), None)
)
trainer.train()

***** Running training *****
  Num examples = 2124
  Num Epochs = 50
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 8
  Total optimization steps = 800
 62%|██████▎   | 500/800 [36:16<21:01,  4.20s/it]  Saving model checkpoint to ./finetuned\checkpoint-500
Configuration saved in ./finetuned\checkpoint-500\config.json


{'loss': 3.2345, 'learning_rate': 3.7974683544303802e-06, 'epoch': 31.24}


Model weights saved in ./finetuned\checkpoint-500\pytorch_model.bin
100%|██████████| 800/800 [51:23<00:00,  2.76s/it]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 800/800 [51:23<00:00,  3.85s/it]

{'train_runtime': 3083.706, 'train_samples_per_second': 34.439, 'train_steps_per_second': 0.259, 'train_loss': 3.074898910522461, 'epoch': 49.96}


TrainOutput(global_step=800, training_loss=3.074898910522461, metrics={'train_runtime': 3083.706, 'train_samples_per_second': 34.439, 'train_steps_per_second': 0.259, 'train_loss': 3.074898910522461, 'epoch': 49.96})

In [9]:
torch.save(model, 'my_model.h5')

In [11]:
model = torch.load('my_model.h5')

In [12]:

text = "С чего начинается человек? "
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model = torch.load('my_model.h5')
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        repetition_penalty=5.0,
                        do_sample=True,
                        use_cache=False,
                        num_beams=2,
                        temperature=1,
                        top_p=0.95,
                        top_k=10,
                        max_length=1000,
                        )

# generated_text = list(map(tokenizer.decode, out))[0]
tokenizer.decode(out[0]).replace('\xa0—', ' ').replace('\n', ' ')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'С чего начинается человек?  С того, что он делает.   – А с чего начинается мужчина? – спросил я.   – Сначала вы начинаете думать о том, как сделать так, чтобы у вас не было никаких обязательств перед кем бы то ни было вообще.  То есть это может быть любой человек, которого вы встретите на своем пути.  Но если вы попытаетесь представить себе эту ситуацию таким образом, вам придется очень долго и мучительно работать над тем, чтобы понять, в чем именно заключается ваша проблема.  Вы должны четко представлять себе, зачем вы идете к этой цели.  И вот тогда-то все и начинается.   – В каком смысле "все"? – спросил я.   – Ну, во-первых, речь идет об ощущении вашей собственной силы. Во-вторых, когда вы выходите за пределы своего поля зрения, ваше внимание сосредоточено на одном предмете или объекте одновременно. Это происходит благодаря тому, что ваш ум постоянно перемещается из одной области сознания в другую. Поэтому при переходе от одного состояния покоя к другому ваши мысли становятся боле

In [13]:
text = "С чего начинается человек? "
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model = torch.load('my_model.h5')
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        repetition_penalty=5.0,
                        do_sample=True,
                        use_cache=False,
                        num_beams=2,
                        temperature=1,
                        top_p=0.95,
                        top_k=10,
                        max_length=1000,
                        )

# generated_text = list(map(tokenizer.decode, out))[0]
tokenizer.decode(out[0]).replace('\xa0—', ' ').replace('\n', ' ')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'С чего начинается человек?  С того, что он делает?   – Ну да. А как иначе? Человек рождается с этим самым «я», и у него есть два пути – или даже три: либо он идет по одному из этих путей, либо он идет по другому… Но все эти варианты не всегда оказываются правильными. Иногда они становятся просто невозможными. И тогда наступает конец света. Вот это самое страшное. Потому что мы живем в такое время, когда каждый новый шаг вперед ведет нас к тому, чтобы стать кем-то другим. То есть перестать быть тем, кем ты был раньше. Стать собой. Понимаете? Это значит сделать то, о чем я говорил выше. Если вы посмотрите на этот график, увидите, что за последние десять тысяч лет человеческая психика претерпела колоссальные изменения. Она стала более гибкой и подвижной, ее стали больше интересовать вопросы морали и порядка, а также социальные проблемы. В общем, жизнь пошла своим чередом. Человеческая психика эволюционировала так быстро, что уже через несколько тысяч лет перестала реагировать на новые ст

In [14]:
text = "С чего начинается человек? "
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model = torch.load('my_model.h5')
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        repetition_penalty=5.0,
                        do_sample=True,
                        use_cache=False,
                        num_beams=2,
                        temperature=1,
                        top_p=0.95,
                        top_k=10,
                        max_length=1000,
                        )

# generated_text = list(map(tokenizer.decode, out))[0]
tokenizer.decode(out[0]).replace('\xa0—', ' ').replace('\n', ' ')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'С чего начинается человек?  С того, что он делает?   – Ну да.  Сначала я думал, что это какой-то сложный механизм, с помощью которого мозг получает информацию из внешнего мира.  Потом я понял, что это не так.  И вот тогда мне показалось, что все эти сложные механизмы работают как-то по-другому.   – То есть вы хотите сказать, что в вашем восприятии мир совсем другой?   – Совершенно точно.  Я даже не могу объяснить, о чем именно идет речь.  Но то, что я увидел, поразило меня до глубины души.  Это было какое-то удивительное космическое путешествие во времени и пространстве.  Как будто мы оказываемся в каком-то параллельном мире одновременно со всеми своими жизненными ситуациями.  Мы словно бы находимся в одном измерении: прошлое, настоящее и будущее сливаются в одно целое.  А потом выясняется, что наш мир совершенно иной.  Вернее, его можно сравнить с микроскопической пылинкой размером с грецкий орех.  Пылинка начинает двигаться к центру нашей галактики, где находится Солнце.  Вокруг Сол

#### Конечно, до полноценного произведения еще долго, но результатом я довольна. Чувствуется рука мастера!